In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle
!cp drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
!kaggle competitions download -c store-sales-time-series-forecasting >> /dev/null
!mkdir store-sales-time-series-forecasting
!!unzip store-sales-time-series-forecasting.zip -d store-sales-time-series-forecasting >> /dev/null

100% 21.4M/21.4M [00:00<00:00, 226MB/s]


[]

In [ ]:
%cd store-sales-time-series-forecasting

/content/store-sales-time-series-forecasting


In [ ]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('dark')

##train.csv
* The training data, comprising time series of features 
store_nbr, family, and onpromotion as well as the target sales.
* store_nbr identifies the store at which the products are sold.
* family identifies the type of product sold.
* sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
* onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

In [ ]:
train_df = pd.read_csv("train.csv")
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
train_dates = pd.DataFrame(train_df.date.unique(), columns=['date'])

##test.csv
* The test data, having the same features as the training data. Need to predict the target sales for the dates in this file.
* The dates in the test data are for the 15 days after the last date in the training data.

In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [ ]:
test_dates = pd.DataFrame(test_df.date.unique(), columns=['date'])

##Create a dataframe for all the dates

In [ ]:
date = train_dates.merge(test_dates, on='date', how='outer')
date.describe()

,date
count,1700
unique,1700
top,2013-01-01
freq,1


##stores.csv
* Store metadata, including city, state, type, and cluster.
* cluster is a grouping of similar stores.

In [ ]:
stores = pd.read_csv("stores.csv")
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


##oil.csv
* Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [ ]:
oil_price = pd.read_csv("oil.csv")
oil_price.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


##Add records for all dates by joining with the date df

In [ ]:
oil_price = date.merge(oil_price, on='date', how='left')
oil_price.isna().sum()

date            0
dcoilwtico    525
dtype: int64

##Fill null values by using next days oil price +- some random values

In [ ]:
oil_price['was_dcoilwtico_nan'] = oil_price.dcoilwtico.isna()
oil_price.dcoilwtico = oil_price.dcoilwtico.fillna(method='bfill')
oil_price.dcoilwtico = oil_price.apply(lambda x: x.dcoilwtico + np.random.uniform(-.25,.25) if x.was_dcoilwtico_nan == True else x.dcoilwtico, axis=1)
oil_price = oil_price.drop('was_dcoilwtico_nan', axis=1)
oil_price

,date,dcoilwtico
0,2013-01-01,92.918372
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.187094
...,...,...
1695,2017-08-27,46.588731
1696,2017-08-28,46.400000
1697,2017-08-29,46.460000
1698,2017-08-30,45.960000


##holidays_events.csv
* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [ ]:
holidays = pd.read_csv("holidays_events.csv")
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


##Create holiday record for all dates by joining with date df

In [ ]:
holidays = date.merge(holidays, on='date', how='outer')
holidays.isna().sum()

date              0
type           1447
locale         1447
locale_name    1447
description    1447
transferred    1447
dtype: int64

In [ ]:
holidays.type.value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Work Day        5
Bridge          5
Name: type, dtype: int64

In [ ]:
from datetime import datetime
def fix(x):

  if x.transferred:
    x['is_holiday'] = False
  elif x.type in ('Bridge', 'Additional', 'Transfer', 'Holiday'):
    x['is_holiday'] = True
  else: x['is_holiday'] = False

  if x.type == 'Event':
    x['is_event'] = True
  else: x['is_event'] = False

  dt = datetime.strptime(x.date, '%Y-%m-%d')

  day = dt.weekday()
  if x.type != 'Work Day' and (day == 5 or day == 6):
    x['is_weekend'] = True
  else:
    x['is_weekend'] = False

  return x[['is_holiday','is_event', 'is_weekend']]
holidays[['is_holiday','is_event', 'is_weekend']] = holidays.apply(fix, axis=1)

In [ ]:
holidays = holidays.drop(columns=['type', 'description', 'locale', 'locale_name', 'transferred'])
holidays.head()

,date,is_holiday,is_event,is_weekend
0,2013-01-01,True,False,False
1,2013-01-02,False,False,False
2,2013-01-03,False,False,False
3,2013-01-04,False,False,False
4,2013-01-05,False,False,False


##transcations.csv

In [ ]:
transactions = pd.read_csv("transactions.csv")
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


##Merge train_df with stores df using the store_nbr column

In [ ]:
train_df = pd.merge(train_df, stores, on='store_nbr', how = 'left')
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [ ]:
test_df = pd.merge(test_df, stores, on='store_nbr', how = 'left')
test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13


##Merge train_df with oil df using the date column

In [ ]:
train_df = pd.merge(train_df, oil_price, on='date', how = 'left')
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,92.918372
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,92.918372
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,92.918372
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,92.918372
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,92.918372


In [ ]:
test_df = pd.merge(test_df, oil_price, on='date', how = 'left')
test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.8
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.8
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.8


##Merge train_df with holidays based on date column

In [ ]:
train_df = pd.merge(train_df, holidays, on='date', how = 'left')
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False


In [ ]:
train_df.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
city           0
state          0
type           0
cluster        0
dcoilwtico     0
is_holiday     0
is_event       0
is_weekend     0
dtype: int64

In [ ]:
test_df = pd.merge(test_df, holidays, on='date', how = 'left')
test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8,False,False,False
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.8,False,False,False
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.8,False,False,False
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8,False,False,False
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.8,False,False,False


##Merge train_df with transactions df on date and store_nbr column

In [ ]:
train_df = pd.merge(train_df, transactions, on=['date','store_nbr'], how = 'left')
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,NaN


In [ ]:
train_df.transactions = train_df.transactions.fillna(0)

In [ ]:
test_df = pd.merge(test_df, transactions, on=['date','store_nbr'], how = 'left')
test_df.transactions = test_df.transactions.fillna(0)
test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.8,False,False,False,0.0
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8,False,False,False,0.0
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0


##Additional Notes
* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [ ]:
from datetime import datetime
import calendar
def calculate(x):
  dt = datetime.strptime(x.date, '%Y-%m-%d')
  next_dt = 0
  if dt.day >= 15:
    next_dt = int(calendar.monthrange(dt.year, dt.month)[1] - dt.day)
  else:
    next_dt = int(15 - dt.day)
  prev_dt = 0
  if dt.day >= 15:
    prev_dt = int(dt.day - 15)
  else:
    prev_dt = int(dt.day - 0)
  x['next_salary_day'] = next_dt
  x['prev_salary_day'] = prev_dt
  return x[['date', 'next_salary_day', 'prev_salary_day']]
salary_days = date.apply(calculate, axis=1)
salary_days

,date,next_salary_day,prev_salary_day
0,2013-01-01,14,1
1,2013-01-02,13,2
2,2013-01-03,12,3
3,2013-01-04,11,4
4,2013-01-05,10,5
...,...,...,...
1695,2017-08-27,4,12
1696,2017-08-28,3,13
1697,2017-08-29,2,14
1698,2017-08-30,1,15


##Merge train_df with salary_days df on date column

In [ ]:
train_df = train_df.merge(salary_days, on='date', how='left')
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,92.918372,True,False,False,0.0,14,1


In [ ]:
test_df = test_df.merge(salary_days, on='date', how='left')
test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1


##Split date into year, month and day

In [ ]:
date['datetime'] = pd.to_datetime(date['date'])

date['year'] = date['datetime'].dt.strftime('%Y')
date['month'] = date['datetime'].dt.strftime('%m')
date['day'] = date['datetime'].dt.strftime('%d')

date.head()

,date,datetime,year,month,day
0,2013-01-01,2013-01-01,2013,01,01
1,2013-01-02,2013-01-02,2013,01,02
2,2013-01-03,2013-01-03,2013,01,03
3,2013-01-04,2013-01-04,2013,01,04
4,2013-01-05,2013-01-05,2013,01,05


##Create a earthquake affected feature

In [ ]:
from datetime import datetime
eq_date = datetime(year=2016, month=4, day=16)
def create(x):
  return (x.datetime - eq_date).days

date['days_a/b_eq'] = date.apply(create, axis=1)
date = date.drop('datetime', axis=1)
date.head()

,date,year,month,day,days_a/b_eq
0,2013-01-01,2013,01,01,-1201
1,2013-01-02,2013,01,02,-1200
2,2013-01-03,2013,01,03,-1199
3,2013-01-04,2013,01,04,-1198
4,2013-01-05,2013,01,05,-1197


##Merge with train_df

In [ ]:
train_df = date.merge(train_df, on='date', how='right')
train_df.head()

,date,year,month,day,days_a/b_eq,id,store_nbr,family,sales,onpromotion,...,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
0,2013-01-01,2013,01,01,-1201,0,1,AUTOMOTIVE,0.0,0,...,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
1,2013-01-01,2013,01,01,-1201,1,1,BABY CARE,0.0,0,...,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
2,2013-01-01,2013,01,01,-1201,2,1,BEAUTY,0.0,0,...,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
3,2013-01-01,2013,01,01,-1201,3,1,BEVERAGES,0.0,0,...,Pichincha,D,13,92.918372,True,False,False,0.0,14,1
4,2013-01-01,2013,01,01,-1201,4,1,BOOKS,0.0,0,...,Pichincha,D,13,92.918372,True,False,False,0.0,14,1


In [ ]:
test_df = date.merge(test_df, on='date', how='right')
test_df.head()

,date,year,month,day,days_a/b_eq,id,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
0,2017-08-16,2017,08,16,487,3000888,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
1,2017-08-16,2017,08,16,487,3000889,1,BABY CARE,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
2,2017-08-16,2017,08,16,487,3000890,1,BEAUTY,2,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
3,2017-08-16,2017,08,16,487,3000891,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1
4,2017-08-16,2017,08,16,487,3000892,1,BOOKS,0,Quito,Pichincha,D,13,46.8,False,False,False,0.0,15,1


#Feature Engineering

In [ ]:
#sns.lineplot(x=train_df['days_a/b_eq'], y=train_df.sales)

In [ ]:
'''from statsmodels.tsa.seasonal import seasonal_decompose


sales = train_df.sales
print(sales.count())
result = seasonal_decompose(sales, model='additive', period=1)
trend = result.trend
seasonality = result.seasonal
residuals = result.resid

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(30,10))

ax1.plot(trend)
ax1.set_title('Trend')

ax2.plot(seasonality)
ax2.set_title('Seasonality')

ax3.plot(residuals)
ax3.set_title('Residuals')

plt.show()'''

"from statsmodels.tsa.seasonal import seasonal_decompose\n\n\nsales = train_df.sales\nprint(sales.count())\nresult = seasonal_decompose(sales, model='additive', period=1)\ntrend = result.trend\nseasonality = result.seasonal\nresiduals = result.resid\n\nfig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(30,10))\n\nax1.plot(trend)\nax1.set_title('Trend')\n\nax2.plot(seasonality)\nax2.set_title('Seasonality')\n\nax3.plot(residuals)\nax3.set_title('Residuals')\n\nplt.show()"

In [ ]:
train_df.describe(include = 'all')

,date,year,month,day,days_a/b_eq,id,store_nbr,family,sales,onpromotion,...,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
count,3054348,3054348,3054348,3054348,3.054348e+06,3.054348e+06,3.054348e+06,3054348,3.054348e+06,3.054348e+06,...,3054348,3054348,3.054348e+06,3.054348e+06,3054348,3054348,3054348,3.054348e+06,3.054348e+06,3.054348e+06
unique,1684,5,12,31,NaN,NaN,NaN,33,NaN,NaN,...,16,5,NaN,NaN,2,2,2,NaN,NaN,NaN
top,2014-06-25,2016,06,25,NaN,NaN,NaN,AUTOMOTIVE,NaN,NaN,...,Pichincha,D,NaN,NaN,False,False,False,NaN,NaN,NaN
freq,7128,670032,286902,110484,NaN,NaN,NaN,92556,NaN,NaN,...,1074678,1018116,NaN,NaN,2676564,2954556,2184732,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,-3.554918e+02,1.504277e+06,2.750000e+01,NaN,3.590209e+02,2.617480e+00,...,NaN,NaN,8.481481e+00,6.778957e+01,NaN,NaN,NaN,1.558656e+03,7.621937e+00,7.623104e+00
std,NaN,NaN,NaN,NaN,4.874346e+02,8.662610e+05,1.558579e+01,NaN,1.107286e+03,1.225494e+01,...,NaN,NaN,4.649735e+00,2.570356e+01,NaN,NaN,NaN,1.036468e+03,4.432367e+00,4.430228e+00
min,NaN,NaN,NaN,NaN,-1.201000e+03,0.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,...,NaN,NaN,1.000000e+00,2.619000e+01,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,-7.770000e+02,7.546768e+05,1.400000e+01,NaN,0.000000e+00,0.000000e+00,...,NaN,NaN,4.000000e+00,4.632000e+01,NaN,NaN,NaN,9.310000e+02,4.000000e+00,4.000000e+00
50%,NaN,NaN,NaN,NaN,-3.535000e+02,1.507572e+06,2.750000e+01,NaN,1.100000e+01,0.000000e+00,...,NaN,NaN,8.500000e+00,5.319000e+01,NaN,NaN,NaN,1.332000e+03,8.000000e+00,8.000000e+00
75%,NaN,NaN,NaN,NaN,6.700000e+01,2.255120e+06,4.100000e+01,NaN,1.960110e+02,0.000000e+00,...,NaN,NaN,1.300000e+01,9.571000e+01,NaN,NaN,NaN,1.980000e+03,1.100000e+01,1.100000e+01


In [ ]:
train_df = train_df.drop(columns = ['date', 'id'])

In [ ]:
cat_cols = train_df.select_dtypes(include=['object'])
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
train_df[cat_cols.columns] = encoder.fit_transform(cat_cols)

train_df.head()

,year,month,day,days_a/b_eq,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
0,0.0,0.0,0.0,-1201,1,0.0,0.0,0,18.0,12.0,3.0,13,92.918372,True,False,False,0.0,14,1
1,0.0,0.0,0.0,-1201,1,1.0,0.0,0,18.0,12.0,3.0,13,92.918372,True,False,False,0.0,14,1
2,0.0,0.0,0.0,-1201,1,2.0,0.0,0,18.0,12.0,3.0,13,92.918372,True,False,False,0.0,14,1
3,0.0,0.0,0.0,-1201,1,3.0,0.0,0,18.0,12.0,3.0,13,92.918372,True,False,False,0.0,14,1
4,0.0,0.0,0.0,-1201,1,4.0,0.0,0,18.0,12.0,3.0,13,92.918372,True,False,False,0.0,14,1


In [ ]:
cat_cols = test_df.select_dtypes(include=['object'])
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
test_df[cat_cols.columns] = encoder.fit_transform(cat_cols)

test_df.head()

,date,year,month,day,days_a/b_eq,id,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,is_event,is_weekend,transactions,next_salary_day,prev_salary_day
0,0.0,0.0,0.0,0.0,487,3000888,1,0.0,0,18.0,12.0,3.0,13,46.8,False,False,False,0.0,15,1
1,0.0,0.0,0.0,0.0,487,3000889,1,1.0,0,18.0,12.0,3.0,13,46.8,False,False,False,0.0,15,1
2,0.0,0.0,0.0,0.0,487,3000890,1,2.0,2,18.0,12.0,3.0,13,46.8,False,False,False,0.0,15,1
3,0.0,0.0,0.0,0.0,487,3000891,1,3.0,20,18.0,12.0,3.0,13,46.8,False,False,False,0.0,15,1
4,0.0,0.0,0.0,0.0,487,3000892,1,4.0,0,18.0,12.0,3.0,13,46.8,False,False,False,0.0,15,1


In [ ]:
def normalize(df: pd.DataFrame)->pd.DataFrame:
  from sklearn.preprocessing import MinMaxScaler, RobustScaler

  scaler = MinMaxScaler()

  for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = scaler.fit_transform(df[col].values.reshape(-1,1))

  return df

In [ ]:
train_df = normalize(train_df)
test_df = normalize(test_df)

In [ ]:
X_train = train_df.drop('sales', axis=1)
y_train = train_df.sales
X_test = test_df.drop(columns = ['date', 'id'])

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=15)
selector.fit(X_train, y_train)

kbest = selector.get_feature_names_out()
kbest

In [ ]:
X_train = X_train[kbest]
X_test = X_test[kbest]

In [ ]:
X_train.select_dtypes(include='object').columns

Index([], dtype='object')

In [ ]:
from xgboost import XGBRegressor, DMatrix

def rmsle(preds, dtrain):
    labels = dtrain.get_label()
    return 'rmsle', np.sqrt(np.mean(np.power(np.log1p(preds) - np.log1p(labels), 2)))

xgboost = XGBRegressor(learning_rate=0.01,n_estimators=200,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror',
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.006, tree_method='gpu_hist', verbosity=2, eval_metric='rmse')
from sklearn.model_selection import cross_val_score
print(cross_val_score(xgboost, X_train, y_train, cv=10, verbose=1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[-147.91921583 -128.54151125  -78.20408175  -71.50723807  -67.41125739
  -52.31413868  -42.46985506  -39.9946723   -36.71964056  -37.11575742]


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.5min finished


In [ ]:
xgboost.fit(X_train, y_train)

XGBRegressor(colsample_bytree=0.7, eval_metric='rmse', learning_rate=0.01,
             min_child_weight=0, n_estimators=200, objective='reg:squarederror',
             reg_alpha=0.006, seed=27, subsample=0.7, tree_method='gpu_hist',
             verbosity=2)

In [ ]:
sales_pred = xgboost.predict(X_test)

In [ ]:
test_df['sales'] = sales_pred

In [ ]:
test_df[['id','sales']].to_csv('submission.csv', index=False)

In [ ]:
!kaggle competitions submit -c store-sales-time-series-forecasting -f submission.csv -m ""

100% 493k/493k [00:02<00:00, 183kB/s]
Successfully submitted to Store Sales - Time Series Forecasting

In [ ]:
!pip install tpot -q

     |████████████████████████████████| 87 kB 4.0 MB/s 
     |████████████████████████████████| 193.6 MB 78 kB/s 
     |████████████████████████████████| 139 kB 76.6 MB/s 


In [ ]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, cv=5, n_jobs=4)
tpot.fit(X_train, y_train)

print("Best hyperparameters:", tpot.fitted_pipeline_)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]